In [1]:
import molsysmt as msm

SyntaxError: invalid syntax (mutate.py, line 29)

In [ ]:
molsys = msm.convert(msm.demo['Met-enkephalin']['vacuum.pdb'], to_form='molsysmt.MolSys')

In [ ]:
molsys.topology.atoms_dataframe['group_id'][68]=3

In [ ]:
msm.get(molsys, element='group', group_id=True)

In [ ]:
msm.get(molsys, element='group', selection='group_name=="MET"', group_index=True)

In [ ]:
aa = {0:1, 1:2}

In [ ]:
for ii,jj in aa.items():
    print(ii,jj)

In [ ]:
molsys = msm.build.mutate(molsys, residue_indices=[1,2], to_residue_names=['ALA', 'VAL'])

In [ ]:
molsys = msm.convert(msm.demo['Met-enkephalin']['vacuum.pdb'], to_form='molsysmt.MolSys')
molsys = msm.build.mutate(molsys, residue_indices=[1,2], to_residue_names=['ALA', 'VAL'])
seq = msm.convert(molsys, to_form='string:aminoacids3')
check = (seq == 'TyrAlaValPheMet')
assert check


In [3]:
aa = 'ALA-37-VAL'

In [4]:
a, b, c = aa.split('-')

In [5]:
a

'ALA'